In [ ]:
# 1) Environment setup
!pip -q install llama-cpp-python codecarbon

!mkdir -p /root/models/mistral
!wget -q -O /root/models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf \
        https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf

# 2) Imports
from codecarbon import EmissionsTracker
from llama_cpp import Llama
import time

# 3) Start emissions tracker (keep the record only in memory)
tracker = EmissionsTracker(project_name="CarbonSwipe-single-run",
                           log_level="error", save_to_file=False)
tracker.start()

# 4) Load the quantised Mistral-7B model (set seed for reproducibility)
llm = Llama(
    model_path="/root/models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    n_gpu_layers=35,          # adjust to your GPU VRAM
    n_ctx=2048,
    seed=42,                  # <<< reproducible sampling
    verbose=False
)

# 5) Prompt and inference
prompt = "Explain how AI can help reduce carbon emissions."
t0 = time.time()
out = llm(prompt, max_tokens=100, stop=["</s>"])
t1 = time.time()

# 6) Stop tracker and fetch metrics
tracker.stop()
energy_kwh = tracker.final_emissions_data.energy_consumed
co2_kg     = tracker.final_emissions_data.emissions

# 7) Display
print(" Model output:\n", out["choices"][0]["text"].strip())
print(f"\n Inference time: {(t1 - t0):.2f} s")
print(f" Energy used:    {energy_kwh * 1_000:.3f} Wh")
print(f" CO₂ emitted:    {co2_kg   * 1_000:.3f} g")

In [ ]:
import csv
import time
from codecarbon import EmissionsTracker
from llama_cpp import Llama
import os

# Load prompts
with open("mnt/Benchmark_Prompts.csv", "r") as f:
    reader = csv.DictReader(f)
    prompts = [row["prompt"] for row in reader]

# Initialize model
llm = Llama(
    model_path="/root/models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    n_gpu_layers=35,
    n_ctx=2048,
    verbose=False
)

# Prepare results file (create if not exists)
results_path = "mnt/benchmark_results.csv"
write_header = not os.path.exists(results_path)

with open(results_path, "a", newline='') as f:
    fieldnames = ["index", "prompt", "output", "inference_time_sec", "energy_Wh", "co2_g"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    if write_header:
        writer.writeheader()

    total = len(prompts)
    for i, prompt in enumerate(prompts[:150]):
        print(f"\n [{i+1}/{total}] Running inference for prompt #{i+1}...")
        tracker = EmissionsTracker(project_name="CarbonSwipe-Benchmark", output_dir=".", log_level="error")
        tracker.start()

        start = time.time()
        output = llm(prompt, max_tokens=150, stop=["</s>"])
        end = time.time()

        tracker.stop()

        # Read last row of emissions
        with open("emissions.csv", "r") as efile:
            emissions_reader = csv.DictReader(efile)
            last = list(emissions_reader)[-1]
            energy_kwh = float(last["energy_consumed"])
            co2_kg = float(last["emissions"])

        inf_time = round(end - start, 2)
        energy_Wh = round(energy_kwh * 1000, 3)
        co2_g = round(co2_kg * 1000, 3)

        # Write to CSV
        writer.writerow({
            "index": i + 1,
            "prompt": prompt,
            "output": output["choices"][0]["text"].strip(),
            "inference_time_sec": inf_time,
            "energy_Wh": energy_Wh,
            "co2_g": co2_g
        })

        # Show live progress
        print(f"⏱ Time: {inf_time:.2f} sec |  Energy: {energy_Wh} Wh |  CO₂: {co2_g} g")
        remaining = total - (i + 1)
        est_left = int((end - start) * remaining)
        print(f" Estimated time remaining: {est_left // 60} min {est_left % 60} sec")
        print("----------------------------------------------------------")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib, textwrap

#  1. Load & enrich data 
CSV_PATH = pathlib.Path("/mnt/benchmark_results.csv")
df = pd.read_csv(CSV_PATH)

# Basic derived columns
df["Output_length"] = df["output"].str.split().str.len()
df["Input_length"]  = df["prompt"].str.split().str.len()
df.rename(columns={"inference_time_sec": "Latency_s"}, inplace=True)

# Optional: prompt-category if you stored it elsewhere
# Here we fall back to a dummy “Unknown” label
if "Category" not in df.columns:
    df["Category"] = "Unknown"

# - 2. Matplotlib / seaborn style 
sns.set_theme(style="whitegrid")
plt.rcParams.update({"figure.dpi": 120})

#  3. Figure 1 – energy histogram 
plt.figure()
sns.histplot(df["energy_Wh"], bins=30, kde=True)
plt.xlabel("Energy per query (Wh)")
plt.ylabel("Count")
plt.title(textwrap.fill("Distribution of energy consumption per query", 40))
plt.tight_layout()
plt.savefig("yo1.png")

#  4. Figure 2 – correlation heat-map 
plt.figure()
corr = df[["energy_Wh", "co2_g", "Latency_s",
           "Input_length", "Output_length"]].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", vmin=-1, vmax=1, square=True)
plt.title("Correlation matrix of key metrics")
plt.tight_layout()
plt.savefig("yo2.png")

#  5. Figure 3 – energy vs output length 
plt.figure()
plt.scatter(df["Output_length"], df["energy_Wh"], alpha=0.6)
plt.xlabel("Output length (tokens)")
plt.ylabel("Energy per query (Wh)")
plt.title("Energy vs. output length")
plt.tight_layout()
plt.savefig("yo3.png")

#  6. Figure 4 – energy vs input length 
plt.figure()
plt.scatter(df["Input_length"], df["energy_Wh"], alpha=0.6)
plt.xlabel("Input length (tokens)")
plt.ylabel("Energy per query (Wh)")
plt.title("Energy vs. input length")
plt.tight_layout()
plt.savefig("yo4.png")

# 7. Figure 5 – box-plot by category 
plt.figure(figsize=(8, 4))
sns.boxplot(x="Category", y="energy_Wh", data=df)
plt.ylabel("Energy per query (Wh)")
plt.title("Energy consumption by prompt category")
plt.tight_layout()
plt.savefig("yo5.png")

# 8. Figure 6 – mean energy bar-plot 
plt.figure(figsize=(8, 4))
df_mean = df.groupby("Category")["energy_Wh"].mean().reset_index()
sns.barplot(x="Category", y="energy_Wh", data=df_mean)
plt.ylabel("Mean energy (Wh)")
plt.title("Mean energy per prompt category")
plt.tight_layout()
plt.savefig("yo6.png")

print(" All plots saved")